<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
# импортируемые библиотеки
import pandas as pd
import psycopg2
import requests
import urllib.parse as req
import lxml

In [5]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [6]:
# текст запроса
query = f'''select
                count(v.id)    -- количество всех вакансий в таблице vacancies
            from vacancies v           
        '''

In [7]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [8]:
# текст запроса
query = f'''select
                count(e.id)    -- количество работодателей в таблице employers
            from employers e           
        '''

In [9]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [10]:
# текст запроса
query = f'''select
                count(a.id)    -- количество регионов в таблице areas
            from areas a           
        '''

In [11]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [12]:
# текст запроса
query = f'''select
                count(i.id)    -- количество сфер деятельности работодателей в таблице industries
                
            from industries i           
        '''

In [13]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,294


***

### Выводы по предварительному анализу данных

Исходя из полученных данных можно грубо оценить некоторые аспекты рынка труда, такие как: 
1) на каждого работодателя приходится примерно по 2 вакансии; 
2) на каждую сферу деятельности приходится около 80 работодателей; 
3) в среднем 17 работодателей приходятся на каждый регион; 
4) около 36 вакансий приходится на каждый регион.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [14]:
# текст запроса
query = f'''select
                count(v.id) cnt,    -- количество вакансий...
                a.name area    -- в каждом регионе
                                    
            from vacancies v    -- объединение таблицы vacancies...
            join areas a on v.area_id=a.id    -- с таблицей areas по id региона
            
            group by a.name    -- группировка по названию региона
            
            order by 1 DESC    -- сортировка по количеству вакансий в порядке убывания            
        '''

In [15]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Тарко-Сале
765,1,Новоаннинский
766,1,Бирск
767,1,Сасово


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [16]:
# текст запроса
query = f'''select
                count(v.id)    -- количество вакансий
                
            from vacancies v
            
            where (v.salary_to is not null) or (v.salary_from is not null)    -- при хотя бы 1 заполненном полем с зарплатой
        '''    

In [17]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [18]:
# текст запроса
query = f'''select
                round(avg(v.salary_from)) down,    -- среднее значение зарплат для нижней границы
                round(avg(v.salary_to)) up    -- среднее значение зарплат для верхней границы
                
            from vacancies v           
        '''    

In [19]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,down,up
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [20]:
# текст запроса
query = f'''select
                count(v.id),    -- количество вакансий
                v.schedule,    -- тип рабочего графика
                v.employment    -- тип трудоустройства
                
            from vacancies v
            
            group by v.schedule, v.employment    -- группировка по типу рабочего графика и типу трудоустройства
            
            order by 1 DESC    -- сортировка по количеству вакансий (1 столбик) в порядке убывания
        '''    

In [21]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [22]:
# текст запроса
query = f'''select
                v.experience    -- требуемый опыт работы      
                
            from vacancies v
            
            group by v.experience    -- группировка по категориям 'опыт работы'
            
            order by count(v.id)    -- сортировка по количеству вакансий в каждой категории опыта работы в порядке возрастания
        '''   

In [23]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

### Выводы по детальному анализу вакансий

Более детальный анализ вакансий на HH.ru позволяет выявить следующие интересные закономерности: 
1) в топ-5 регионов по количеству вакансий на HH.ru входят такие города, как Москва, Санкт-Петербург, Минск, Новосибирск, Алматы — большие города, много вакансий; 
2) средние значения зарплатной вилки 71065 и 110537 рублей (нижняя и верхняя граница соответственно), можно отметить, что нижняя и верхняя усредненные границы не сильно друг от друга отличаются; 
3) наиболее распространенная комбинация графика с типом занятости — полный день и полная занятость, наименее распространенная — смешанный график и проекстная работа; 
4) самое распространенное требование к опыту работы — 1-3 года, самое нераспростроненное — более 6 лет (таких специалистов немного, к тому же они требуют большую оплату труда). 

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [24]:
# текст запроса
query = f'''(-- создание первой строки: работодатель на первом месте по количеству вакансий
            select
                e.name    -- название работодателя       
                
            from employers e    -- объединение таблицы employers...
            join vacancies v on v.employer_id=e.id    -- с таблицей vacancies по id работодателя
            
            group by e.name    -- группировка по имени 
            
            order by count(v.id) DESC    -- сортировака по количеству вакансий в порядке убывания
            
            limit 1)    -- выводим только 1 значение (работодатель с наибольшим количеством вакансий)
            
            union    -- объединение полученного результата со  следующим
            
            (-- создание второй строки: работодатель на пятом месте по количеству вакансий
            select
                e.name    -- название работодателя     
                
            from employers e    -- объединение таблицы employers...
            join vacancies v on v.employer_id=e.id    -- с таблицей vacancies по id работодателя
            
            group by e.name    -- группировка по имени
            
            order by count(v.id) DESC    -- сортировака по количеству вакансий в порядке убывания
            
            offset 4 limit 1)    -- отсечение первых 4х названий и последующих за первым после offset (5-ое место) 
        '''

In [25]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name
0,Газпром нефть
1,Яндекс


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
# текст запроса

#Запрос для первой части задания: таблица с количеством работодателей и вакансий в каждом регионе
query_1 = f'''select
                a.name region,
                count(e.id) employer_cnt,
                count(v.id) vacancie_cnt  
                
            from areas a
            left join vacancies v on v.area_id=a.id
            left join employers e on a.id=e.area
                                   
            group by a.name
            
            order by 2 DESC                 
                
        '''
 
#Запрос для второй части задания:  регион с наибольшим количеством работодателей и без вакансий       
query_2 = f'''select
                a.name region,    -- название региона
                count(e.id) employer_cnt,    -- количество работодателей
                count(v.id) vacancie_cnt    -- количество вакансий
                
            from areas a    -- объединение таблицы areas...
            left join vacancies v on v.area_id=a.id    -- с таблицей vacancies ...
            left join employers e on a.id=e.area    -- и с таблицей employers по идентификатору региона
                                   
            where v.id is null    -- при условии отсутствия информации о вакансиях
            
            group by a.name    -- группировка по названию региона
            
            order by 2 DESC    -- сортировка по количеству работодателей (2й столбец) в порядке убывания
            
            limit 1    -- ограниченичение: только 1е значение 
                
        '''    

In [27]:
# результат запроса
df_2 = pd.read_sql_query(query_2, connection)
df_2   #выводим ответ на вторую часть задания

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\732841490.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql_query(query_2, connection)


,region,employer_cnt,vacancie_cnt
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [28]:
# текст запроса
query = f'''select
                e.name,    -- название работодателя
                count(distinct v.area_id)    -- количество регионов (без повторений)
                
            from employers e    -- объединение таблицы employers...
            join vacancies v on v.employer_id=e.id    -- с таблицей vacancies по id работодателя             
                   
            group by e.name    -- группировка по названию работодателя
            
            order by 2 DESC    -- сортировка по количеству регионов (2й столбец) в порядке убывания
                  
        '''  

In [29]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
# текст запроса
query = f'''select
                count(e.id)    -- количество работодателей
                
            from employers e    -- объединение таблицы employers...
            left join employers_industries ei on e.id=ei.employer_id    -- с таблицей employers_industries по идентификатору работодателя
            
            where ei.industry_id is null    -- при условии отсутствия информации о сфере деятельности                         
                
        '''

In [31]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [32]:
# текст запроса
query = f'''select
                e.name    -- название работодателя
                
            from employers e    -- объединение таблицы employers...
            left join employers_industries ei on e.id=ei.employer_id    -- с таблицей employers_industries по id работодателя
            
            group by e.name    -- группировка по имени работодателя
            
            having count(ei.industry_id)=4    -- при наличии 4х сфер деятельности
            
            order by 1    -- сортировка по названию работодателя (1й столбец)
            
            offset 2 limit 1    -- вывод третьей строки      
                
        '''


In [33]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [34]:
# текст запроса
query = f'''select
                count(e.id)    -- количество работодателей
                
            from employers e    -- объединение таблицы employers...
            left join employers_industries ei on e.id=ei.employer_id    -- с таблицей employers_industries по id работодателя...
            left join industries i on i.id=ei.industry_id    -- и таблицей industries по id сферы деятельности
            
            where i.name='Разработка программного обеспечения'    -- при сфере деятельности 'Разработка программного обеспечения'
                                         
        '''


In [35]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [36]:
# код для получения списка городов-милионников

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')    # разбор URL-адреса на компоненты
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)    # адаптированный URL
cities = tuple(pd.read_html(url)[1]['Город'])    # кортеж городов из таблицы на сайте

In [37]:
# текст запроса
query = f'''with t as    -- создание таблицы из запроса

        (select
                a.name,    -- название региона
                count(v.id) cnt    -- количество вакансий
                
            from areas a    -- объединение таблицы areas...
            left join vacancies v on a.id=v.area_id    -- с таблицей vacancies по id региона...
            left join employers e on e.id=v.employer_id    -- и таблицей employers по id работодателя
            
            -- при условии, что название работодателя 'Яндекс' и название региона есть в списке городов-миллионников
            where e.name='Яндекс' and a.name in {cities}
            
            group by a.name)    -- группировка по названию региона
            
        (select
                t.name,    --название региона
                t.cnt    -- количество вакансий
            
            from t)    -- из созданной нами таблицы
            
        union    -- объединение результата предыдущего запроса со следующим
        
        (select
                'Total',    -- создание строки 'Total'
                sum(t.cnt)    -- сумма вакансий
                
            from t)    -- из созданной нами таблицы
            
        order by cnt    -- сортируем объединенный запрос по возрастанию количества вакансий 
                                         
        '''

In [38]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

### Выводы по анализу работодателей

Анализ работодателей показал следующие особенности: 
1) на сайте HH.ru первое место по количеству вакансий принадлежит компании Газпром нефть; 
2) регион, в котором наибольшее количество работодателей без вакансий — Россия, возможно это связано с особенностью заполнения анкет; 
3) Яндекс публикует резюме в наибольшем количестве регионов (181), что модет говорить о широком распространении предооставления услуг; 
4) 8419 работодателей не публикует информацию о сфере своей деятельности; 
5) 3553 работодателей относятся к разработке программнго обеспечения; 
6) В Москве по сравнению с другими городами-миллионниками наибольшее количество вакансий, публикуемых компанией Яндекс (54), общее количество вакансий в городах-миллионниках — 485.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [39]:
# текст запроса

query = f'''select
                count(v.id)    -- количество вакансий
                
            from vacancies v
            
            -- при наличчии фрагментов слов 'data' и 'данн' вне зависимости от регистра            
            where v.name ilike '%data%' or v.name ilike '%данн%'    
                                         
        '''


In [40]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [41]:
# текст запроса

query = f'''select
                count(v.id)    -- количество вакансий
                
            from vacancies v
            
            -- для вакансий дата-сайентиста уровня Junior        
            where (v.name ilike '%data scientist%' or
            v.name ilike '%data science%' or
            v.name ilike '%исследователь данных%' or
            (v.name like '%ML%' and v.name not like '%HTML%') or
            v.name ilike '%machine learning%' or
            v.name ilike '%машинн%обучен%') and
            (v.name ilike '%junior%' or
            v.experience ilike '%нет опыта%' or
            v.employment ilike '%cтажтровка%')
        '''


In [42]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [43]:
# текст запроса
query = f'''select
                count(v.id)    -- количество вакансий
                
            from vacancies v
            
            -- для дата-сайентистов с ключевым навыком SQL или postgres           
            where (v.name ilike '%data scientist%' or
            v.name ilike '%data science%' or
            v.name ilike '%исследователь данных%' or
            (v.name like '%ML%' and v.name not like '%HTML%') or
            v.name ilike '%machine learning%' or
            v.name ilike '%машинн%обучен%') and
            (v.key_skills ilike '%sql%' or
            v.key_skills ilike '%postgres%')
                                         
        '''

In [44]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [45]:
# текст запроса
query = f'''select
                count(v.id)    -- количество вакансий
                
            from vacancies v
            
            -- для вакансий дата-сайентиста с ключевым навыком Python           
            where (v.name ilike '%data scientist%' or
            v.name ilike '%data science%' or
            v.name ilike '%исследователь данных%' or
            (v.name like '%ML%' and v.name not like '%HTML%') or
            v.name ilike '%machine learning%' or
            v.name ilike '%машинн%обучен%') and
            (v.key_skills ilike '%python%')
                                         
        '''

In [46]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [47]:
# текст запроса
query = f'''select
                -- среднее количество ключевых навыков
                round(avg(((length(v.key_skills)+1)-length(replace(v.key_skills,CHR(9),'')))),2)
                
            from vacancies v
            
            -- для дата-сайентистов
            where v.name ilike '%data scientist%' or
            v.name ilike '%data science%' or
            v.name ilike '%исследователь данных%' or
            (v.name like '%ML%' and v.name not like '%HTML%') or
            v.name ilike '%machine learning%' or
            v.name ilike '%машинн%обучен%'
                                         
        '''

In [48]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [49]:
# текст запроса
query = f'''select
                v.experience,    -- опыт работы
                -- средняя зарплата с учетом недостаточности информации о зарплате
                round(avg(coalesce((v.salary_from+v.salary_to)/2,v.salary_from,v.salary_to))) avg_salary
                
            from vacancies v
            
            -- для дата сайентиста           
            where (v.name ilike '%data scientist%' or
            v.name ilike '%data science%' or
            v.name ilike '%исследователь данных%' or
            (v.name like '%ML%' and v.name not like '%HTML%') or
            v.name ilike '%machine learning%' or
            v.name ilike '%машинн%обучен%')
            
            group by v.experience    -- группировка по опыту работы                       
        '''

In [50]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,experience,avg_salary
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

### Выводы по предметному анализу

Исходя из полученных данных можно сделать следующие выводы: 
1) дата-сайентисты с ключевым навыком 'Python' в примерно в 1.75 раз более востребованы, чем с ключевым навыком 'SQL' или 'Postgres'; 
2) среднее количество навыков дата-сайентиста - 6.41, что говорит о том, что работодателей интересуют специалисты, умеющие решать разноплановые задачи; 
3) примерно по 75 тысяч в среднем обещают работодатели 51 Junior дата-сайентисту; 
4) опытным DS специалистам (от 3 до 6 лет) работодатели обещают зарплату более 200 тысяч, а специалисты, опыт которых превышает 6 лет, не фигурируют в списках вакансий; 
5) уже начиная с годового опыта работы в сфере DS зарплата повышается почти в 2 раза.

# Общий вывод по проекту

Исходя из тех данных, которые были получены в результате работы над проектом, можно сделать несколько общих выводов: 
1) исходя из количества анкет, опубликованных на hh.ru, покрытия по регионам вакансиий и количества работадателей, можно сделать вывод о популярности сайта и о  возможностям посетителей сайта найти работу; 
2) и не просто найти работу, а найти хорошо оплачиваемую работу (средняя зарплатная вилка от 71065 и 110537 рублей, за 2023 средняя зарплата в России - 73 709 руб); 
3) дата-сайенс специалисты востребованы, и работодатели хорошо  оплачивают их работу, особеннно если те имеют большое число навыков в этой области.

# Дополнительные исследования данных

1. Сколько вакансий с типом рабочего графика 'Удаленная работа' есть в таблице vacancies?

In [51]:
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования
# текст запроса
query = f'''select
                count(v.id)    -- количество вакансий
                            
            from vacancies v
            
            where v.schedule = 'Удаленная работа'    -- при типе рабочего графика 'Удаленная работа'
            
        '''           

In [52]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,9311


2. Топ-10 работадателей по среднему размеру оклада, минимальная и максимальная зарплата, которую они предлагают

In [53]:
# текст запроса
query = f'''select
                e.name,    -- имя работодателя
                -- средняя зарплата с учетом недостаточности информации о зарплате
                round(avg(coalesce((v.salary_from+v.salary_to)/2,v.salary_from,v.salary_to))) avg_salary,
                min(v.salary_from),    -- минимальная зарплата
                max(v.salary_to)    --  максимальная зарплата
                
            from employers e    -- объединение таблицы employers с...
            join vacancies v on e.id = v.employer_id    -- таблицей vacancies по id работодателя
            
            -- при условии, что заполнено хотя бы 1 поле с зарплатой            
            where v.salary_from is not null and v.salary_to is not null
            
            group by e.name     -- группировка по имени работодателя
                                   
            order by 2 DESC    -- сортировка по средней зарплате (2 столбец, от большего к меньшему)
            
            limit 10    -- ограничение первыми 10 строками
            
        '''

In [54]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name,avg_salary,min,max
0,ЦТП,650000.0,350000,950000
1,RuWork,555261.0,175346,935176
2,Петухова Алия,552700.0,217000,860000
3,Агентство недвижимости ЛЕТО,550000.0,200000,1000000
4,Пикабу,550000.0,500000,600000
5,BeRecruited,550000.0,500000,600000
6,Шеф Маркет,457500.0,130000,1000000
7,think-cell,452751.0,348270,557232
8,Итон,450000.0,200000,700000
9,BrainShells,438363.0,292242,584485


3. В каком регионе ищет сотрудника работодатель с максимальным средним окладом и какие ключевые навыки он требует?

In [55]:
query = f'''select 
                a.name    -- название региона
                
            from employers e    -- объединение таблицы employers с...
            join areas a on e.area = a.id    -- таблицей areas по id региона
                    
            where e.name = 'ЦТП'    -- для работадателя ЦТП
            
        union    -- объединение предыдущего запроса со следующим
        
            select
                v.key_skills    -- ключевые навыки
                
            from vacancies v    -- объединение таблицы vacancies с...            
            join employers e on e.id = employer_id    -- таблицей employers по id работодателя
            
            where e.name = 'ЦТП'    -- для работодателя ЦТП                      
                                                         
        '''

In [56]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,name
0,PostgreSQL\tMS SQL Server\t1С программирование
1,Москва


### 4. Какое количество вакансий размещено для дата-сайенс специалистов?

In [57]:
# текст запроса
query = f'''select
                count(v.id)    -- количество вакансий
                
            from vacancies v    -- из таблицы vacancies
            
            --        
            where (v.name ilike '%data scientist%' or
            v.name ilike '%data science%' or
            v.name ilike '%исследователь данных%' or
            (v.name like '%ML%' and v.name not like '%HTML%') or
            v.name ilike '%machine learning%' or
            v.name ilike '%машинн%обучен%')
        '''

In [58]:
# результат запроса
df = pd.read_sql_query(query, connection)
df

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12868\1115395102.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,count
0,480


### Выводы по дополнительным исследованиям

Дополнительные исследовавания позволяют расширить выводы из предыдущих юнитов: 
1) около 19% резюме (использовались данные, полученные в задании 3.1)   имеют тип рабочего графика 'Удаленая работа', вероятно, такая большая цифра явилась следствием постковида и ростом востребованности IT-специалистов, вероятно, в будущем эта цифра будет становиться больше; 
2) работодатель с самой большой средней зарплатой (650000.0) ищет сотрудника в Москве и требует от него навыки: PostgreSQL, tMS SQL Server, t1С программирование; 
4) топ-10 работодателей по средней зарплате готовы предложить зарплату от 130000 до 1000000;
5) из вакансий, связанных с данными, около 27% дата-сайентистов (использовались данные, полученные в задании 6.1); около 42% востребованных дата-сайентистов должны иметь навык SQL или Postgres (использовались данные, полученные в задании 6.3),  около 73% - навык Python (использовались данные, полученные в задании 6.4)

In [59]:
# закрытие соединения после окончания работы
connection.close()